# 比较MXNet和TensorFlow的基础操作
需要安装的库如下：
- MXNet
- TensorFlow
- Numpy

In [1]:
import mxnet as mx
import tensorflow as tf
import numpy as np

首先比较的是MXNet和TensorFlow的张量生成，还有比较输出形式
* mx.nd.array和tf.Variable不接受shape作为输入参数，而tf.constant接受

In [2]:
a=mx.nd.array([1,2,3])
b=mx.nd.array([[1,2,3],[2,3,4]])
print(a.shape,b.shape)
{'a.shape':a.shape,'b.shape':b.shape}

(3,) (2, 3)


{'a.shape': (3,), 'b.shape': (2, 3)}

In [3]:
a=tf.constant([3.,3.],shape=(1,2))
b=tf.constant([[2.],[2.]])
print(a,b)
c=a*b
print(c)
with tf.Session() as sess:
    print(sess.run(c))

Tensor("Const:0", shape=(1, 2), dtype=float32) Tensor("Const_1:0", shape=(2, 1), dtype=float32)
Tensor("mul:0", shape=(2, 2), dtype=float32)
[[ 6.  6.]
 [ 6.  6.]]


In [4]:
a=tf.constant([1,2,3],shape=(1,3))
b=tf.constant([[1,2,3],[2,3,4]])
print(a,b)
a=tf.Variable([1,2,3],name='a')
b=tf.Variable([[1.,2,3],[2,3,4]],name='b')
c=tf.Variable([[1.,2,3],[2,3,4]],name='c')
d=b+c
print(a,'\n',b,'\n',c,'\n',d,'\n')
with tf.Session() as sess:
    init=tf.global_variables_initializer()
    #init=tf.initialize_all_variables()
    sess.run(init)
    print(sess.run(d))

Tensor("Const_2:0", shape=(1, 3), dtype=int32) Tensor("Const_3:0", shape=(2, 3), dtype=int32)
<tf.Variable 'a:0' shape=(3,) dtype=int32_ref> 
 <tf.Variable 'b:0' shape=(2, 3) dtype=float32_ref> 
 <tf.Variable 'c:0' shape=(2, 3) dtype=float32_ref> 
 Tensor("add:0", shape=(2, 3), dtype=float32) 

[[ 2.  4.  6.]
 [ 4.  6.  8.]]


接下来比较MX和TF创建变量时能否接受类型
- MXNet在创建张量时就能够接受类型
- tf.Variable不能接受类型，类型根据初始值来定，且不可改变，通过相应函数改变类型会使变量变成张量

In [6]:
# float32 is used by default
a = mx.nd.array([1,2,3])
# create an int32 array
b = mx.nd.array([1,2,3], dtype=np.int32)
# create a 16-bit float array
c = mx.nd.array([1.2, 2.3], dtype=np.float16)
(a.dtype, b.dtype, c.dtype,d.dtype)

(numpy.float32, numpy.int32, numpy.float16, tf.float32)

In [7]:
a = tf.Variable(tf.eye(2,2),name='a')
b = tf.Variable(tf.zeros([2,3],dtype=tf.float16),name='b')
c = tf.to_int32(a)
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run([a,b,c]))
    a=tf.to_double(a)
    print(sess.run(a))
    print(a,b,c)

[array([[ 1.,  0.],
       [ 0.,  1.]], dtype=float32), array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float16), array([[1, 0],
       [0, 1]])]
[[ 1.  0.]
 [ 0.  1.]]
Tensor("ToDouble:0", shape=(2, 2), dtype=float64) <tf.Variable 'b_1:0' shape=(2, 3) dtype=float16_ref> Tensor("ToInt32:0", shape=(2, 2), dtype=int32)


MXNet有数种方法可以产生placeholder的效果，而tf也可以通过tf.variable或者tf.placeholder完成placeholder的效果。但是MXNet的用法更像是平时C或者Matlab里变量的用法，即什么都可以干，可以是variable，也可以是constant，也可以是placeholder
- tf.placeholder可以指定shape也可以不指定，像神经网络里时可以指定列数但不指定行数
- tf.placeholder在使用是一定要feed数据，否则会报错
- tf.placeholder不能接受一个tf.tensor对象，必须是python对象、numpy或者tensorhandle
- mx和tf中数据类型不同时均不能直接计算

In [8]:
# create a 2-dimensional array full of zeros with shape (2,3)
a = mx.nd.zeros((2,3))
# create a same shape array full of ones
b = mx.nd.ones((2,3))
# create a same shape array with all elements set to 7
c = mx.nd.full((2,3), 7)
# create a same shape whose initial content is random and
# depends on the state of the memory
d = mx.nd.empty((2,3))

In [9]:
a = tf.placeholder(tf.float16,shape=(None,3))
b = np.random.random([2,3])
print(type(b))
c = tf.to_float(a)+b
with tf.Session() as sess:
    print(sess.run(c,feed_dict={a:b}))

<class 'numpy.ndarray'>
[[ 1.47177196  1.9306283   0.38896868]
 [ 1.50954461  0.00949893  0.92321837]]
